In [1]:
!nvidia-smi

Thu Mar  6 23:22:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 PCIe               Off |   00000000:2A:00.0 Off |                    0 |
| N/A   43C    P0             87W /  350W |    4741MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
!pip install -q datasets torch wandb

In [ ]:
# !pip install -q torch

In [ ]:
# !pip install wandb

In [3]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_api = user_secrets.get_secret("wandb_api")
wandb_api = "e7058844a75b9e97963d352a1396ab8a09e0c2bd"
import wandb
wandb.login(key=wandb_api)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ragkaushik2004 (ragkaushik2004-vit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
from datasets import load_dataset
dataset = load_dataset("Hemanth-thunder/tamil-open-instruct-v1", split="train")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
text = dataset.features['text']

In [6]:
text

Value(dtype='string', id=None)

In [7]:
dataset

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 493813
})

In [8]:
dataset['text'][0]

'சரியான பதிலுடன் வேலையை வெற்றிகரமாக முடிக்க, வழங்கப்பட்ட வழிகாட்டுதல்களைப் பின்பற்றி, தேவையான தகவலை உள்ளிடவும்.\n\n### Instruction:\nஇலக்கணம், வாக்கிய அமைப்பு மற்றும் தெளிவு ஆகியவற்றை மேம்படுத்த பின்வரும் பத்தியை மறுபரிசீலனை செய்யவும். \n\n### Input:\nநகரங்களில் வாழ்க்கை இன்று மிகவும் அழுத்தமாக இருக்கலாம். போக்குவரத்து நெரிசல் மற்றும் நிரம்பிய பொது போக்குவரத்து மக்களை வாக்குவாதத்திற்கும் சண்டைக்கும் வழிவகுக்கும். மன மற்றும் உடல் ஆரோக்கியத்தை பாதிக்கும் மற்றொரு பிரச்சனை மாசு. சமாளிப்பதற்கான வழிகளைக் கண்டுபிடிப்பது முக்கியம்.\n\n### Response:\nநகரங்களில் வாழ்க்கை இந்த நாட்களில் மிகவும் அழுத்தமாக உள்ளது. போக்குவரத்து நெரிசல் மற்றும் நெரிசலான பொது போக்குவரத்து வாக்குவாதங்கள் மற்றும் மோதல்களுக்கு வழிவகுக்கும். மாசுபாடு என்பது மன மற்றும் உடல் ஆரோக்கியத்தை எதிர்மறையாக பாதிக்கும் மற்றொரு பிரச்சினை. சமாளிப்பதற்கான வழிகளைக் கண்டுபிடிப்பது முக்கியம்.'

In [9]:
!pip install -q transformers trl peft sentencepiece

In [ ]:
# !pip install sentencepiece

used sentencepiece coz got this error, ValueError: Cannot instantiate this tokenizer from a slow version. If it's based on sentencepiece, make sure you have sentencepiece installed.

In [10]:
!pip install --upgrade -q transformers

In [11]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1", use_fast = True)
tokenizer.pad_token = tokenizer.eos_token

# def tokenize_function(example):
#     prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
#     return tokenizer(prompt, padding="max_length", truncation=True, max_length=1000)

dataset_train_subset = dataset.select(range(5000))
dataset_test_subset = dataset.select(range(5000, 6000))


# tokenized_dataset = dataset_subset.map(tokenize_function, batched=True)
# tokenized_dataset = tokenized_dataset.remove_columns(["instruction", "input", "output"])
# tokenized_dataset.set_format("torch")
# print(tokenized_dataset)


## Checking cache accumulation

In [12]:
!pip install -q psutil

In [36]:
import psutil

# Get cache memory information
virtual_memory = psutil.virtual_memory()
print(f"Cache Memory: {virtual_memory.cached / (1024**2):.2f} MB")


Cache Memory: 55608.67 MB


In [37]:
# import psutil

# Get cache memory and convert to GB
virtual_memory = psutil.virtual_memory()
cache_memory_gb = virtual_memory.cached / (1024**3)  # Convert bytes to GB

print(f"Cache Memory: {cache_memory_gb:.2f} GB")


Cache Memory: 54.31 GB


## Instruction-tuning Sarvam

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1", fast_tokenizer = True)

# Example usage
text = "कर्नाटक की राजधानी है:"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=5)
result = tokenizer.decode(outputs[0])


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.91it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
print(tokenizer.is_fast)  # Should return True if it's a fast tokenize

True


In [17]:
result 

'<s> कर्नाटक की राजधानी है: बैंगलोर।\n'

In [18]:
dataset

Dataset({
    features: ['instruction', 'output', 'input', 'text'],
    num_rows: 493813
})

In [19]:
from transformers import AutoTokenizer

model_name = "sarvamai/sarvam-1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



In [20]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # LoRA for causal language modeling
    inference_mode=False,  # Set to True if using the model for inference
    r=16,  # LoRA rank (lower is more efficient)
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.05  # Dropout to prevent overfitting
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.62it/s]


trainable params: 3,211,264 || all params: 2,528,299,008 || trainable%: 0.1270


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./sarvam-lora-finetuned",
    eval_strategy="epoch",  # Updated parameter name
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"]
)

trainer.train()



In [22]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType


# train_dataset = tokenized_dataset.range(10000)
# eval_dataset = tokenized_dataset.range(10000, 12000)

split = int(10000)
train_dataset = Dataset.from_dict(tokenized_dataset[:split])
eval_dataset = Dataset.from_dict(tokenized_dataset[10000:12000])

model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./sarvam-lora-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.53it/s]


trainable params: 3,211,264 || all params: 2,528,299,008 || trainable%: 0.1270


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

In [23]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    
    # Set input_ids as labels (shift left for Causal LM training)
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized


In [24]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 493813/493813 [02:21<00:00, 3493.50 examples/s]


In [25]:
# train_dataset = tokenized_dataset
# eval_dataset = tokenized_dataset

train_dataset = Dataset.from_dict(tokenized_dataset[:10000])
eval_dataset = Dataset.from_dict(tokenized_dataset[10000:12000])

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.333300,1.003922
2,0.293200,0.994902
3,0.333100,0.994566


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=3750, training_loss=0.3663211066563924, metrics={'train_runtime': 1545.515, 'train_samples_per_second': 19.411, 'train_steps_per_second': 2.426, 'total_flos': 2.20155346944e+17, 'train_loss': 0.3663211066563924, 'epoch': 3.0})

## Clear Cache

In [ ]:
%reset -f

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)


In [ ]:
import shutil

hf_cache = "~/.cache/huggingface"
shutil.rmtree(hf_cache, ignore_errors=True)
print("Hugging Face cache cleared")


In [ ]:
import torch
torch.cuda.empty_cache()
print("Cleared GPU cache")

In [6]:
!jcache project clear 

/bin/bash: line 1: jcache: command not found


In [8]:
!pip install -q jupyter_cache

In [9]:
from pathlib import Path
import nbformat as nbf
from jupyter_cache import get_cache
from jupyter_cache.base import CacheBundleIn
from jupyter_cache.executors import load_executor, list_executors
from jupyter_cache.utils import (
    tabulate_cache_records, 
    tabulate_project_records
)


In [10]:
cache = get_cache(".jupyter_cache")
cache.clear_cache()
cache

JupyterCacheBase('/workspace/.jupyter_cache')

In [11]:
print(cache.list_cache_records())
print(cache.list_project_records())

[]
[]


## Using the finetuned model

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "sarvamai/sarvam-1"  
lora_model_path = "./sarvam-lora-finetuned/checkpoint-3750/"  

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

model = PeftModel.from_pretrained(base_model, lora_model_path)

print("Fine-tuned LoRA model loaded successfully!")


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.07s/it]


Fine-tuned LoRA model loaded successfully!


In [31]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

instruction = """### Instruction:
தமிழில் ஒரு கவிதை எழுதவும்.

### Response:"""

output = generator(instruction, max_length=200)

print("\n🎯 Generated Response:\n", output[0]['generated_text'])


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausa


🎯 Generated Response:
 ### Instruction:
தமிழில் ஒரு கவிதை எழுதவும்.

### Response:
கவிதை:
இலையுதிர் காலத்தின் மாலை நேரத்தில்,
பனிமூட்டமான வானம்,
காற்று மெதுவாக வீசுகிறது,
அதன் மென்மையான தொடுதல்,
இலைகள் விழும் சத்தம்,
ஒரு அமைதியான, அமைதியான சூழல்,
உலகம் தூங்குகிறது,
ஆனால் என் இதயம் துடிக்கிறது,
என் இதயத்தின் ஆழத்தில்,
நான் உன்னை நினைக்கிறேன், என் அன்பே,
நீ என்னுடன் இருக்கிறாய், என் இதயம்,
எப்போதும், எப்போதும்,
நினைவுகள், கனவுகள் மற்றும் அன்பின் நினைவுகள்,
ஒவ்வொரு நொடியும், ஒவ்வொரு நொடியும்,
நான் உன்னை நினைக்கிறேன், என் அன்பே,
நீ என்னுடன் இருக்கிறாய், என் இதயம்,
எப்போதும், எப்போதும்,
நினைவுகள், கனவுகள்


### Uploading to huggingface_hub

In [38]:
!pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [39]:
!pip install -U "huggingface_hub[cli]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli login --token ""

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `sarvam_wrtie_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `sarvam_wrtie_token`


In [44]:
from huggingface_hub import HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_username = "raghz"
model_name = "sarvam-instruct-tuned-tamil"

repo_id = f"{hf_username}/{model_name}"


model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"✅ Model uploaded successfully! View it at: https://huggingface.co/{repo_id}")


adapter_model.safetensors: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12.9M/12.9M [00:03<00:00, 3.98MB/s]
tokenizer.model: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.94M/1.94M [00:02<00:00, 877kB/s]


✅ Model uploaded successfully! View it at: https://huggingface.co/raghz/sarvam-instruct-tuned-tamil


## Inference from hf

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

repo_id = "raghz/sarvam-instruct-tuned-tamil"

tokenizer = AutoTokenizer.from_pretrained(repo_id)
model = AutoModelForCausalLM.from_pretrained(repo_id)

input_text = "### Instruction:\nதமிழில் ஒரு கவிதை எழுதவும்.\n\n### Response:"
inputs = tokenizer(input_text, return_tensors="pt")
output = model.generate(**inputs, max_length=150)

print("\n Generated Response:\n", tokenizer.decode(output[0], skip_special_tokens=True))


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.35it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
